# Importing Dataset
<p><img src="1.png" alt="Entity Relationships Diagram">
<ul>Note: 
<li>Each order can have more than one point (pick-up or drop-off). Each point has an address, coordinates, the number of the point in the route, the start and end time of the interval when the courier should arrive at the point.</li> 
<li>	Status field in contracts table: status=0 means active, 1 - canceled, 2 - completed. Status_last_update_date - the date and time of the contract status last update.</li>
</ul>


In [2]:
import sqlalchemy
import sqlite3
import pandas as pd
import numpy as np
%load_ext sql

In [3]:
%%sql
sqlite:///dostavista.db

In [4]:
contracts = %sql SELECT * FROM contracts;
contracts = contracts.DataFrame()

points = %sql SELECT point_id, order_id, sequence, delivery_interval_start, delivery_interval_end, address FROM points;
points = points.DataFrame()

orders = %sql SELECT * FROM orders;
orders = orders.DataFrame()

 * sqlite:///dostavista.db
Done.
 * sqlite:///dostavista.db
Done.
 * sqlite:///dostavista.db
Done.


# Cleaning Dataset
Following DataFrames require <code>to_datetime</code> conversions. And <code>to_numeric</code> conversions.

In [9]:
# Converting columns to datetime
orders.iloc[:, -2:] = orders.iloc[:, -2:].apply(pd.to_datetime, errors='coerce', infer_datetime_format=True)
points.iloc[:, [3, 4]] = points.iloc[:, [3, 4]].apply(pd.to_datetime, errors='coerce', infer_datetime_format=True)
contracts.iloc[:, -4:] = contracts.iloc[:, -4:].apply(pd.to_datetime, errors='coerce', infer_datetime_format=True)

# Converting columns to numeric
orders.dropna(inplace=True)
orders[['courier_id', 'contract_id']] = orders[['courier_id', 'contract_id']].astype('int')

In [12]:
display('contracts', contracts, 'orders', orders, 'points', points)

'contracts'

,contract_id,courier_id,status,status_last_update_date,contract_start,contract_end,created_date
0,1,1,1,2019-11-09 19:00:00,2019-11-09 11:00:00,2019-11-09 19:00:00,2019-11-09 15:00:00
1,2,2,1,2019-11-09 19:00:00,2019-11-09 11:00:00,2019-11-09 19:00:00,2019-11-09 15:00:00
2,3,3,1,2019-11-09 19:00:00,2019-11-09 11:00:00,2019-11-09 19:00:00,2019-11-09 15:00:00
3,4,4,0,2019-11-11 19:00:00,2019-11-09 11:00:00,2019-11-09 19:00:00,2019-11-09 15:00:00


'orders'

,order_id,courier_id,user_id,contract_id,created,courier_assignment_time
0,1,1,21,1,2019-11-09 07:14:00,2019-11-09 07:14:00
2,3,3,19,3,2019-11-09 00:48:00,2019-11-09 00:48:00
4,5,4,16,4,2019-11-11 11:36:00,2019-11-11 11:36:00
5,6,4,24,4,2019-11-11 11:10:00,2019-11-11 11:36:00
6,7,4,24,4,2019-11-11 11:10:00,2019-11-11 11:36:00
7,8,4,23,4,2019-11-11 17:46:00,2019-11-11 11:36:00


'points'

,point_id,order_id,sequence,delivery_interval_start,delivery_interval_end,address
0,1,1,0,2019-11-09 07:14:00,2019-11-09 12:00:00,"Москва, Дубининская, 1"
1,2,1,1,2019-11-09 07:14:00,2019-11-09 18:00:00,"Москва, Дубининская, 2"
2,3,2,0,2019-11-09 06:00:00,2019-11-09 09:00:00,"Москва, Дубининская, 3"
3,4,2,1,2019-11-09 16:14:00,2019-11-09 19:00:00,"Москва, Дубининская, 4"
4,5,3,0,2019-11-09 00:48:00,2019-11-09 02:00:00,"Москва, Дубининская, 5"
5,6,3,1,2019-11-09 00:48:00,2019-11-09 01:00:00,"Москва, Дубининская, 6"
6,7,3,2,2019-11-09 00:48:00,2019-11-09 02:00:00,"Москва, Дубининская, 7"
7,8,4,0,2019-11-12 11:38:00,2019-11-12 13:00:00,"Москва, Дубининская, 8"
8,9,4,1,2019-11-12 11:38:00,2019-11-12 13:00:00,"Москва, Дубининская, 9"
9,10,5,0,2019-11-21 11:36:00,2019-11-21 14:00:00,"Москва, Дубининская, 10"


# Task 1
### Question:
Our rules prohibit the courier to work in two modes simultaneously. Why: because if a courier commits to work during contract working hours and is doing at least one freelance order with overlapping delivery intervals, it can lead to delays and cancellations of orders. Write SQL script that finds examples of couriers, orders, and contracts that violate this rule. 

### Solution:
Assuming that <code>sequence</code> column describes order of delivery drop offs (I do not possess full documentation of data variables), we can observes that some deliveries do not make sense. I can complete pairwise comparison of preceding elements of the DataFrame to detect asynchronous time intervals.

In [93]:
grouped_df = points.drop(columns='address').groupby('order_id')

col = 'delivery_interval_end'
points['conflict'] = grouped_df.apply(lambda x: x[col] < x[col].shift()).reset_index(drop=True)

points

,point_id,order_id,sequence,delivery_interval_start,delivery_interval_end,address,conflict
0,1,1,0,2019-11-09 07:14:00,2019-11-09 12:00:00,"Москва, Дубининская, 1",False
1,2,1,1,2019-11-09 07:14:00,2019-11-09 18:00:00,"Москва, Дубининская, 2",False
2,3,2,0,2019-11-09 06:00:00,2019-11-09 09:00:00,"Москва, Дубининская, 3",False
3,4,2,1,2019-11-09 16:14:00,2019-11-09 19:00:00,"Москва, Дубининская, 4",False
4,5,3,0,2019-11-09 00:48:00,2019-11-09 02:00:00,"Москва, Дубининская, 5",False
5,6,3,1,2019-11-09 00:48:00,2019-11-09 01:00:00,"Москва, Дубининская, 6",True
6,7,3,2,2019-11-09 00:48:00,2019-11-09 02:00:00,"Москва, Дубининская, 7",False
7,8,4,0,2019-11-12 11:38:00,2019-11-12 13:00:00,"Москва, Дубининская, 8",False
8,9,4,1,2019-11-12 11:38:00,2019-11-12 13:00:00,"Москва, Дубининская, 9",False
9,10,5,0,2019-11-21 11:36:00,2019-11-21 14:00:00,"Москва, Дубининская, 10",False


### Answer:
By finding conflicting delivery end periods in <code>points</code> tables, we can reference <code>order_id</code> to fetch appropriate <code>courier_id</code> from <code>orders</code> table.

In [100]:
confl_couriers = orders[orders['order_id'].isin(points[points['conflict']==True]['order_id'])]
confl_couriers
print('couriers with conflicting delivery_dates have ids:{}'.format(set(confl_couriers.courier_id)))

couriers with conflicting delivery_dates have ids:{3, 4}


# Task 2.
### Questions:
<li>•	What are the advantages and disadvantages of having more than one working mode in such a Company? </li>
<li>•	What would you offer as a Product Analyst for monitoring if the balance between these working modes is normal and doesn’t require any manual actions from the Operations Team? What would be the main metrics in such a dashboard for monitoring? What features would you offer to improve metrics that you mentioned?</li>

### Answers:
<ul>
    <li> The simplest way to track balance between freelance and contract workforce is to create new feature categorizeing every <code>courier_id</code> with <code>freelance</code> or <code>contract</code> labels. And then dividing aggregate numbers of both to derive a ratio. It is also easy to set up a Piechart in Tableau displaying said ratio.</li>
    <li> Biggest advantage of having workforce comprised of freelancers is reduced cost structure of payroll. Yet the biggest drawback is high employee turnover paired with unpredictable workforce's size </li>
    <li> Suppose supply of freelancers dries up during customers' orders hike, without proper absorption mechanism the only option is to increase compensations.</li>
    <li> Assuming that workforce demand flactuates based on 4 factors Trend factors, Cyclical factors, Seasonal factors, and Irregular factors. I would recommend to sustain steady size of contract workers capable absorbing workforce shortages. To achieve such goals I recommend to use Manpower Forecasting using Time-Series Analysis.</li>